<style>
    div.output_area{
        max-height:600px;
        overflow:scroll;
    }
</style>


[RAPID2018](https://rapid2018.org/)


# [RAMP](https://www.ramp.studio/problems/mars_craters) on Primary Vertex reconstruction



1. [Introduction](#Introduction)
2. [Preprocessing](#Preprocessing)
3. [Workflow](#Workflow)
4. [Evaluation](#Scoring)
5. [Local testing/exploration](#Local-testing)
6. [Submission](#Submitting-to-ramp.studio)

## Introduction


### The prediction task



### References



## Preprocessing



With no further do, let's have a look at the data.

### Required dependencies and downloads

#### Installation of libraries and `ramp-workflow`

To get this notebook running and test your models locally using the `ramp_test_submission`, we recommend that you use the Python distribution from [Anaconda](https://www.anaconda.com/download/) or [Miniconda](https://conda.io/miniconda.html). 

In [2]:
# !conda env create -f environment.yml   # use the environment.yml file to create the 'mars-craters' env
# !source activate mars-craters          # activate the virtual environment

**OR** if you have Python already installed but are **not using Anaconda**, you'll want to use `pip` 

In [3]:
# !pip install -r requirements.txt

#### Download script (optional)

If the data has not yet been downloaded locally, uncomment the following cell and run it.

There are ~700Mb of images.

In [4]:
# !python download_data.py

#### The input data
The input data consists of a json file for each event (where an event is a bunch crossing).
The json file contains the hits in the Velo detector, Velo states from a simplified Kalman filter of the reconstructed Velo tracks, as well as the MC truth information.

In [5]:
import json
from matplotlib import pyplot as plt

In [6]:
jdata = json.load(open('data/train/RapidVPData_6719289_74665.json'))
print (X_train)

NameError: name 'X_train' is not defined

We are interested in the reconstructed hits and tracks

In [ ]:
VeloTracks  = jdata['VeloTracks']
VeloHits  = jdata['VPClusters']
print(VeloTracks)
print(VeloHits)

The hits are made of (x,y,z) coordinates:

In [ ]:
for key, item in VeloHits.items():
    print (item['x'], item['y'], item['z'])

We can, for example, pltos the x-y distribution of all hits:

In [ ]:
hits_x = [item['x'] for  key, item in VeloHits.items()]
hits_y = [item['y'] for  key, item in VeloHits.items()]
plt.plot(hits_x,hits_y, 'o', markersize=1.)

or z distribution

In [ ]:
hits_z = [item['z'] for  key, item in VeloHits.items()]
plt.hist(hits_z)

For submitting at the [RAMP site](http://ramp.studio), you will have to write an `ObjectDetector` class in a file named `object_detector.py` that will handle the training and prediction .

### Object detector

A dummy object detector which detects a single crater in the middle of each patch.

Format of the data it receives:

In [ ]:
import problem

In [ ]:
X, y = problem.get_train_data('.')

In [ ]:
X.shape

A single image patch:

In [ ]:
print(X)

There are a couple of helper classes to illustrate how the data is organized.
The test and train data X is a numpy array of object of type EventData. Let' have a closer look at this helper class:

In [ ]:
event = X[0]

It contains the two members, hits and tracks, which are lists containing the Velo hits and states, respectively, again using helper classes:

In [ ]:
event.hits

In [ ]:
event.tracks

The VeloHit class just holds the x,y,z coordinates of the hit:

In [ ]:
hit = event.hits[0]
print (hit)
print(hit.x, hit.y, hit.z)

The VeloTrack helper class holds the state (x,y,z,tx,ty,p/q) of a simplified Kalman fit of the track, as well as the corresponding covariance matrix:

In [ ]:
track = event.tracks[0]
print(track)
print("track state:")
print("x:", track.x)
print("y:", track.y)
print("z:", track.z)
print("tx:", track.tx)
print("ty:", track.ty)
print("pq:", track.pq)

print("covariance matrix:")
print("cov x:", track.cov_x)
print("cov y:", track.cov_y)
print("cov tx:", track.cov_tx)
print("cov ty:", track.cov_ty)
print("cov xtx:", track.cov_xtx)



again, we can make plots:

In [ ]:
hits_x = [hit.x for hit in event.hits]
hits_y = [hit.y for hit in event.hits]
hits_z = [hit.z for hit in event.hits]

In [ ]:
plt.plot(hits_x, hits_y, 'o')

In [ ]:
plt.hist(hits_z)

It should be noted that the p/q is always zero, since we have no momentum estimate from the Velo Kalman fit.
The covariance matrix consist of the diagonal elements x, y, tx, ty and the off-diagonal element xtx. The Velo Kalman fit contains some simplifications, resulting that the covariance element for x and y is always equal. The same is true for the xtx and yty elements